# Tarea NLP — Exploración, Preprocesamiento, TF‑IDF y Clasificación

Este notebook responde a las preguntas solicitadas usando los archivos:

- `df_train.csv`
- `df_test.csv`

> **Nota sobre NLTK**: para evitar dependencias de descargas (stopwords/punkt/wordnet), el preprocesamiento usa `stop_words='english'` de **scikit‑learn**, que es estándar en pipelines TF‑IDF.


In [ ]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import Counter
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS, TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, accuracy_score

# Paths (ajusta si cambias la ubicación de los archivos)
TRAIN_PATH = "df_train.csv"
TEST_PATH  = "df_test.csv"

train = pd.read_csv(TRAIN_PATH)
test  = pd.read_csv(TEST_PATH)

print("train shape:", train.shape)
print("test  shape:", test.shape)
train.head()

## 1) Concatenar ambos conjuntos de datos (mirada general)

Concatenamos **train + test** en un solo DataFrame (`df_all`) para explorar distribución de clases, largos de oraciones y frecuencia de palabras.


In [ ]:
df_all = pd.concat([train.assign(split="train"), test.assign(split="test")], ignore_index=True)
df_all[['split','label','tweet']].head()

### Funciones auxiliares de limpieza y tokenización

- Se remueven **URLs**, **@mentions** y entidades HTML del tipo `&#12345;`
- Se dejan solo letras y apóstrofes (por ejemplo, `don't`, `i'm`)
- Se pasa a minúsculas
- Para análisis de palabras, se filtran **stopwords** y tokens muy cortos (`len <= 2`)


In [ ]:
url_re = re.compile(r'https?://\S+|www\.\S+')
mention_re = re.compile(r'@\w+')
html_entity_re = re.compile(r'&#\d+;')
nonword_re = re.compile(r"[^a-zA-Z'\s]+")

def clean_text_basic(s: str) -> str:
    s = str(s)
    s = url_re.sub(' ', s)
    s = mention_re.sub(' ', s)
    s = html_entity_re.sub(' ', s)
    s = nonword_re.sub(' ', s)
    s = re.sub(r'\s+', ' ', s).strip().lower()
    return s

def tokenize_words(s: str):
    s = clean_text_basic(s)
    toks = re.findall(r"[a-zA-Z']+", s)
    toks = [t for t in toks if (t not in ENGLISH_STOP_WORDS and len(t) > 2)]
    return toks

def count_words(s: str) -> int:
    s = clean_text_basic(s)
    return len(re.findall(r"[a-zA-Z']+", s))

df_all['tweet_clean'] = df_all['tweet'].map(clean_text_basic)
df_all['n_words'] = df_all['tweet'].map(count_words)

df_all[['tweet','tweet_clean','n_words']].head()

## 1.1 Número de ejemplos por tipo de clase — ¿Está balanceado?

Se grafica el conteo por clase usando el dataset concatenado (`df_all`).


In [ ]:
class_counts = df_all['label'].value_counts().sort_index()
class_names = {0: "hate_speech", 1: "offensive", 2: "neither"}

display(class_counts.rename(index=class_names).to_frame("count"))

plt.figure()
plt.bar([class_names[i] for i in class_counts.index], class_counts.values)
plt.title("Número de ejemplos por clase (train + test)")
plt.ylabel("Cantidad de ejemplos")
plt.xticks(rotation=0)
plt.show()

total = class_counts.sum()
print("\nProporciones:")
for k,v in class_counts.items():
    print(f"- {class_names[k]} ({k}): {v} ({v/total:.2%})")

print("\nComentario:")
print("El dataset NO está balanceado: la clase 'offensive' domina ampliamente.")

## 1.2 Largo de las oraciones por clase (número de palabras) — ¿Hay patrones?

Calculamos el número de palabras por tweet (`n_words`) y comparamos distribuciones por clase.


In [ ]:
summary = (df_all
           .groupby('label')['n_words']
           .agg(['count','mean','median','std','min','max'])
           .rename(index=class_names))
display(summary)

# Boxplot por clase
data = [df_all.loc[df_all['label']==k, 'n_words'].values for k in [0,1,2]]
plt.figure()
plt.boxplot(data, labels=[class_names[k] for k in [0,1,2]], showfliers=False)
plt.title("Distribución del largo (número de palabras) por clase")
plt.ylabel("Número de palabras")
plt.show()

print("Comentario:")
print("- En este dataset, 'neither' tiende a tener oraciones un poco más largas (mayor mediana/promedio).")
print("- 'hate_speech' y 'offensive' son muy similares en longitud, por lo que el largo por sí solo no separa bien esas clases.")

## 1.3 Top 40 palabras más frecuentes por clase — comentario

Para cada clase:
1) limpiamos el texto,
2) tokenizamos,
3) removemos stopwords,
4) contamos frecuencias.

> Se reportan las 40 palabras más frecuentes y se comenta el resultado.


In [ ]:
top_words = {}
for k in [0,1,2]:
    toks = []
    for t in df_all.loc[df_all['label']==k, 'tweet']:
        toks.extend(tokenize_words(t))
    top_words[k] = Counter(toks).most_common(40)

for k in [0,1,2]:
    print(f"\n=== Clase {k}: {class_names[k]} ===")
    for w,c in top_words[k]:
        print(f"{w:15s} {c}")

print("\nComentario (general):")
print("- 'hate_speech' y 'offensive' comparten mucho vocabulario ofensivo, lo que puede producir confusiones.")
print("- 'neither' incluye más palabras contextuales (p.ej. temas generales), aunque puede contener términos ofensivos también.")


## 2) Preprocesamiento del texto (y justificación)

Como el objetivo es construir una matriz **TF‑IDF** y luego entrenar un clasificador clásico de ML, usamos un preprocesamiento que reduzca ruido:

- **Lowercasing**: reduce duplicados (`Dog` vs `dog`).
- Remover **URLs**, **@mentions** y entidades HTML: suelen no aportar a la semántica de clase.
- Mantener solo letras y apóstrofes: evita tokens basura.
- `stop_words='english'`: reduce palabras extremadamente frecuentes y poco informativas.
- `ngram_range=(1,2)`: unigrams + bigrams capturan expresiones cortas (útiles en toxicidad/hate).
- `min_df=2` y `max_df=0.9`: filtra términos muy raros y demasiado comunes.

> En un enfoque más avanzado (Transformers), normalmente se haría mucho menos preprocesamiento y se usaría el tokenizer del modelo.


## 3) Construir matriz TF‑IDF (train y test) y tamaño del vocabulario

- `fit` **solo** con `train`
- `transform` para `test`


In [ ]:
vectorizer = TfidfVectorizer(
    preprocessor=clean_text_basic,
    stop_words='english',
    ngram_range=(1,2),
    min_df=2,
    max_df=0.9
)

X_train = vectorizer.fit_transform(train['tweet'])
X_test  = vectorizer.transform(test['tweet'])

vocab_size = len(vectorizer.vocabulary_)

print("X_train shape:", X_train.shape)
print("X_test  shape:", X_test.shape)
print("Tamaño del vocabulario (train):", vocab_size)

## 4) Entrenar un clasificador de Machine Learning

Se entrena un **Linear SVM (LinearSVC)**, que suele funcionar bien con TF‑IDF en texto.

Como el dataset está desbalanceado, usamos `class_weight='balanced'` para penalizar más los errores en clases minoritarias.


In [ ]:
clf = LinearSVC(class_weight='balanced', random_state=42, dual=False)
clf.fit(X_train, train['label'])

pred = clf.predict(X_test)
acc = accuracy_score(test['label'], pred)
print("Accuracy (test):", round(acc, 4))

## 5) Métricas: Precision, Recall y F1-score (general y por clase)

Se reportan métricas por clase y promedios macro/weighted.
- **Macro avg**: promedia clases por igual (útil con desbalance).
- **Weighted avg**: pondera por soporte (tiende a favorecer la clase mayoritaria).


In [ ]:
print(classification_report(
    test['label'], pred,
    target_names=[class_names[0], class_names[1], class_names[2]],
    digits=4
))

print("Comentario:")
print("- Normalmente la clase minoritaria ('hate_speech') obtiene menor recall/precision.")
print("- 'offensive' tiende a ser más fácil por su alta cantidad de ejemplos.")
print("- Mirar macro-F1 es clave para evaluar desempeño equilibrado entre clases.")

## 6) Matriz de confusión e interpretación

La matriz muestra:
- Filas: clase real
- Columnas: clase predicha


In [ ]:
cm = confusion_matrix(test['label'], pred, labels=[0,1,2])
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[class_names[0], class_names[1], class_names[2]])

plt.figure()
disp.plot(values_format='d')
plt.title("Matriz de confusión (test)")
plt.show()

print("Matriz (filas=real, cols=pred):\n", cm)

print("\nInterpretación:")
print("- Un error común es confundir 'hate_speech' con 'offensive' por vocabulario muy similar.")
print("- Los aciertos altos en 'offensive' pueden inflar el accuracy debido al desbalance.")
